In [1]:
import pandas as pd
import re
df = pd.read_excel('오토큐_수정임시.xlsx')

In [3]:
# 괄호와 괄호 안의 텍스트를 모두 제거하는 함수
def remove_parentheses_and_content(text):
    return re.sub(r'\(.*?\)', '', text)

# 쉼표와 쉼표 뒤의 모든 텍스트를 제거하는 함수
def remove_text_after_comma(text):
    return re.split(r',', text)[0]

# '주소'와 '업체명' 열에 대해 괄호와 괄호 안의 내용 제거
df['주소'] = df['주소'].apply(remove_parentheses_and_content)
df['업체명'] = df['업체명'].apply(remove_parentheses_and_content)

# '주소' 열에서 쉼표와 그 뒤의 텍스트 제거
df['주소'] = df['주소'].apply(remove_text_after_comma)

# '검색어' 열 생성 - '주소'와 '업체명'을 결합
df['검색어'] = df['주소'] + ' ' + df['업체명']

# '검색어' 열만 남기기
df = df[['검색어']]

df

,검색어
0,서울 송파구 송이로 149 동부모터스
1,서울 송파구 동남로18길 41-1 가락점 기아오토큐
2,서울 구로구 남부순환로105길 80 가리봉동 가리봉점 기아오토큐
3,서울 서대문구 증가로 251 가좌점 기아오토큐
4,서울 은평구 서오릉로 221 갈현점 기아오토큐
...,...
763,충북 청주시 흥덕구 사운로386번길 45 북청주서비스 기아오토큐
764,충북 청주시 흥덕구 월명로201번길 2 봉명동 공단공업사
765,충북 청주시 흥덕구 풍산로48번길 14 가경동 가경점 기아오토큐
766,충북 충주시 계명대로 185 연수동 연수점 기아오토큐


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# Initialize data storage
data = []

def fetch_naver_map_data(keyword):
    # Initialize driver
    driver = webdriver.Chrome()
    driver.get("https://map.naver.com/p/")

    # Search keyword
    search_box = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "input_search")))
    search_box.clear()
    search_box.send_keys(keyword)
    search_box.send_keys(Keys.ENTER)
    time.sleep(3)  # Adjust if necessary

    # Check if place suggestion appears after the search
    try:
        place_suggestion = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "div.end_inner.place"))
        )
        first_place_button = place_suggestion.find_element(By.CSS_SELECTOR, "button.link_space")
        first_place_button.click()
        time.sleep(2)
    except:
        pass  # If the place suggestion section is not found, continue without interaction

    # Switch to iframe to start gathering information
    try:
        iframe = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "entryIframe")))
        driver.switch_to.frame(iframe)
        
        # Fetch required fields with None as default if elements are missing
        try:
            name = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.LylZZ span.GHAhO"))).text
        except:
            name = None

        try:
            rating_text = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "div.dAsGb span.PXMot.LXIwF"))
            ).text.split("별점")[-1].strip()
            rating = float(rating_text) if rating_text else None
        except:
            rating = None

        try:
            visitor_reviews = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "span.PXMot a[href*='review/visitor']"))
            ).text.split()[-1]
        except:
            visitor_reviews = None

        try:
            blog_reviews = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "span.PXMot a[href*='review/ugc']"))
            ).text.split()[-1]
        except:
            blog_reviews = None

        # Expand and collect operating hours
        try:
            expand_hours_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, "//div[contains(@class, 'A_cdD')]/following-sibling::span[@class='_UCia']"))
            )
            expand_hours_button.click()
            time.sleep(1)
            hours_elements = driver.find_elements(By.CSS_SELECTOR, "div.w9QyJ div.y6tNq")
            open_hours = [element.text for element in hours_elements] if hours_elements else None
        except:
            open_hours = None

        # Locate and click the "리뷰" tab
        try:
            review_tab = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//div[@class='flicking-camera']//a[span[text()='리뷰']]"))
            )
            review_tab.click()
            time.sleep(2)
            
            # Expand the keywords section for "이런점이 좋았어요"
            try:
                more_button = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, "a.dP0sq")))
                more_button.click()
                time.sleep(2)
            except:
                pass

            # Collect keywords and counts from "이런점이 좋았어요"
            keywords = {}
            for element in driver.find_elements(By.CSS_SELECTOR, "li.MHaAm"):
                try:
                    keyword_text = element.find_element(By.CSS_SELECTOR, "span.t3JSf").text
                    count_text = element.find_element(By.CSS_SELECTOR, "span.CUoLy").text
                    count = int(count_text.split()[-1])
                    keywords[keyword_text] = count
                except:
                    keywords[keyword_text] = None
        except:
            keywords = None

        # Collect 댓글1, 댓글2, 댓글3 information
        comments = []
        try:
            # Scroll to load comments
            for _ in range(3):  # Scroll 3 times or until all comments are collected
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(2)  # Wait for comments to load

                # Locate and collect the first three comments
                comment_elements = driver.find_elements(By.CSS_SELECTOR, "div.pui__vn15t2 a[data-pui-click-code='rvshowmore']")
                if len(comment_elements) >= 3:
                    comments = [comment.text for comment in comment_elements[:3]]
                    break
                elif len(comment_elements) > 0:
                    comments = [comment.text for comment in comment_elements]
                    comments += [None] * (3 - len(comments))
                    break
            comments += [None] * (3 - len(comments))  # Ensure exactly 3 comments or None if fewer
        except:
            comments = [None, None, None]

        # Append collected data
        data.append({
            "검색어": keyword,
            "업체명": name,
            "별점": rating,
            "방문자 리뷰": visitor_reviews,
            "블로그 리뷰": blog_reviews,
            "영업 시간": open_hours,
            "이런점이 좋았어요": keywords,
            "댓글1": comments[0],
            "댓글2": comments[1],
            "댓글3": comments[2]
        })

    except Exception as e:
        print(f"Error fetching data for {keyword}: {e}")

    finally:
        driver.quit()

# Assuming df is your DataFrame with keywords
for keyword in df['검색어']:
    fetch_naver_map_data(keyword)

# Convert collected data to DataFrame
result_df = pd.DataFrame(data)
result_df

In [43]:
result_df.to_excel('오토큐_네이버_최종.xlsx')